# Features from the paper

`Easley, D., López de Prado, M., O’Hara, M., Zhang, Z., 2020. Microstructure in the Machine Age. Rev. Financ. Stud. 34, 3316–3363`

In [ ]:
# import some packages


In [1]:
# Load the data


## Features

### 1. Roll measure

## 2. ...